# Age Classifier with Deep Learning

**Описание проекта:**
Целью этого проекта была разработка и реализация модели для классификации возраста на основе изображений. Данные были разделены на наборы для обучения и тестирования, где каждый файл изображения представлял одну из четырёх возрастных категорий (0, 1, 2, 3). Основная цель заключалась в создании точной модели, которая способна предсказывать возрастные категории на основе входных изображений, а также оптимизировать её для эффективного использования.

### **1-шаг**: распаковка файла

In [ ]:
import zipfile
import os

path = '/content/data.zip'
expath = '/content/'

with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall(expath)


### **2-шаг**: preprocessing.

0. У нас имеются изображения с "битыми" именами, их пропускаем (можно просто переименовать), дабы получить нужное нам количество классов. Их 4.
1. Сортируем картинки по классам и перемещаем их в соответсвующие папки.
2. Делаем обработку проще с переводом изображений в черно-белое с CV2.
3. Стандартизируем размер изображений (228, 228).

In [ ]:
import os
import cv2

def preprocess_images(source_dir, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    class_folders = ['0', '1', '2', '3']
    for class_folder in class_folders:
        class_path = os.path.join(output_folder, class_folder)
        if not os.path.exists(class_path):
            os.makedirs(class_path)

    moved_files = 0
    for file_name in os.listdir(source_dir):
        age_class = file_name.split('_')[0]
        if age_class in class_folders:
            source_path = os.path.join(source_dir, file_name)
            destination_path = os.path.join(output_folder, age_class, file_name)

            image = cv2.imread(source_path)
            if image is not None:
                resized_image = cv2.resize(image, (224, 224))
                grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                cv2.imwrite(destination_path, grayscale_image)
                moved_files += 1
            else:
                print(f"Failed to read image: {file_name}")
        else:
            print(f"Ignored file: {file_name}")

    print(f"Moved and processed {moved_files} files from {source_dir} to {output_folder}")


train_dir = '/content/train/'
test_dir = '/content/test/'

prep_train1 = '/content/prep_train1'
prep_test1 = '/content/prep_test1'

preprocess_images(train_dir, prep_train1)
preprocess_images(test_dir, prep_test1)

print("completed")

Ignored file: 2?_4566.jpeg
Ignored file: 0nan_5555.jpg
Moved and processed 10097 files from /content/train/ to /content/prep_train1
Ignored file: 0???.jpg
Moved and processed 101 files from /content/test/ to /content/prep_test1
completed


### **3-шаг**: работа с моделью

Подключаем **Tensorflow**.

Используем более-менее известную модель **VGG19** (**EfficientB0** показал себя чуть хуже).

0. Удаляем верхний слой модели для применения/адаптации своих настроек/слоев (ImageNet к примеру). Поэтому пишем `include_top=False`.

1. Фиксируем веса модели, чтобы во время тренировки они не менялись.
`base_model.trainable = False`

2. Компилируем модель с оптимизатором `adam`, функцией потерь `categorical_crossentropy` и метрикой `accuracy`. Оптимизатор `adamW` не дал отличительных результатов.

3. **ImageDataGen** отвечает за искусственное увеличение данных для компенсирования относительно маленьких датасетов.

4. Генерируем дополнительные тестовые и тренировочные данные, для дальнейшей работы с метриками.

**ModelCheckpoint** помогает нам сохранить лучшую модель по `val_accuracy`, **EarlyStopping** для предотвращения оверфиттинга, сравнимо с коробкой передач (в нашем случае стопаем после "безрезультатных" 10 эпох по `val_loss`)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

input_shape = (224, 224, 3)

base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

base_model.trainable = False

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/prep_train1',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/prep_test1',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

checkpoint = ModelCheckpoint('/content/best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

base_model.trainable = True
for layer in base_model.layers[:15]:
    layer.trainable = False


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

model.save('/content/model_fine_tuned.h5')

80134624/80134624 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d_1  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 20026436 (76.39 MB)
Trainable params: 2052 (8.02 KB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________
Found 10097 

### **4-шаг**: testing

Тестируем модель на входных данных: загружаем изображение и стандартизируем. Выводим результат.

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

loaded_model = tf.keras.models.load_model('/content/model_fine_tuned.h5')

img_path1 = '/content/nazik.PNG'
img = image.load_img(img_path1, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

prediction = loaded_model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)
print(f'Predicted сlass: {predicted_class}')

1/1 [==============================] - 1s 778ms/step
Predicted сlass: [3]


### **5-шаг**: переводим модель в ONNX формат

In [ ]:
from tensorflow.keras.models import load_model
import onnx
import tf2onnx

model = load_model('/content/model_fine_tuned.h5')

onnx_model_path = "/content/model_fine_tuned.onnx"
spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(onnx_model_path, "wb") as f:
    f.write(model_proto.SerializeToString())

print(f"Model saved to {onnx_model_path}")

Model saved to /content/model_fine_tuned.onnx


### **6-шаг**: пишем примитивный интерфейс для нашей модели

Anvil - платформа, которая дает возможность написать
интерфейс на Python. Мы можем связать нашу модель с сервером и запустить. Код самого интерфейса прилагается.

Приложение доступно по [ссылке](https://testovoepredict.anvil.app/), нужно лишь загрузить готовую модель в колаб и запустить код ниже:  

In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server
import onnxruntime as ort
import numpy as np
import cv2

anvil.server.connect("server_Q4MJELTUXJQKASIXWVD54MOD-LHRD3LZOPXWXBF2S")

onnx_model_path = '/content/model_fine_tuned.onnx'
ort_session = ort.InferenceSession(onnx_model_path)

def preprocess_image(file):
    file_bytes = file.get_bytes()
    np_arr = np.frombuffer(file_bytes, np.uint8)
    image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    resized_image = cv2.resize(image, (224, 224))
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    normalized_image = grayscale_image / 255.0
    reshaped_image = np.expand_dims(normalized_image, axis=-1)
    input_image = np.repeat(reshaped_image, 3, axis=-1)
    input_image = np.expand_dims(input_image, axis=0).astype(np.float32)
    return input_image

@anvil.server.callable
def predict_item(file):
    input_image = preprocess_image(file)
    ort_inputs = {ort_session.get_inputs()[0].name: input_image}
    ort_outs = ort_session.run(None, ort_inputs)
    predicted_class = np.argmax(ort_outs[0])
    return str(predicted_class)

anvil.server.wait_forever()

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('/content/model_fine_tuned.h5')

# Predict the labels for the test set
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:\n", cm)

# Classification report
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())
print("Classification Report:\n", report)

# Plot confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

ValueError: Layer "dense_1" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, name=keras_tensor_33>, <KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, name=keras_tensor_34>]